In [1]:
#middle value between best and worst 
(2.377262100000e+07 + 2.131835800000e+07)/2

22545489.5

In [2]:
from fullplan_best import fullplan_best
from fullplan_merit import fullplan_merit
from fullplan_worst import fullplan_worst

In [3]:
#最优
fullplan_best=fullplan_best()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-07-31
Set parameter MIPGap to value 1e-07
Set parameter TimeLimit to value 300
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 253474 rows, 240824 columns and 835436 nonzeros
Model fingerprint: 0x6d087f31
Variable types: 0 continuous, 240824 integer (240824 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 6e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+01]
Presolve removed 96794 rows and 94704 columns
Presolve time: 2.11s
Presolved: 156680 rows, 146120 columns, 523815 nonzeros
Variable types: 0 continuous, 146120 integer (146120 binary)
Found heuristic solution: objective 6.402188e+07

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...


Root simplex log...

Iteration    Objective       Primal 

In [57]:
#中间方案
fullplan_merit=fullplan_merit()

In [74]:
#全整包
fullplan_worst=fullplan_worst()

In [95]:
fullplan_best['fullplan'] = fullplan_best['fullplan'].apply(frozenset)
fullplan_merit['fullplan'] = fullplan_merit['fullplan'].apply(frozenset)
fullplan_worst['fullplan'] = fullplan_worst['fullplan'].apply(frozenset)
# assume df1, df2, df3 are your three pandas DataFrames
# to find common rows among all three dataframes
common_rows = pd.merge(fullplan_best, fullplan_merit, how='inner')
common_rows = pd.merge(common_rows, fullplan_worst, how='inner')

In [149]:
#至始至终没有动过的打包方案
common_rows

,code,fullplan,cnt,FastRequest,NormalRequest,extra
0,node_01_A,"((node_01_A, node_11_C))",1,0,1121,0.0
1,node_01_A,"((node_01_A, node_04_C))",1,0,1716,0.0
2,node_01_A,"((node_01_A, node_04_A))",1,0,3819,0.0
3,node_01_A,"((node_01_A, node_09_B))",1,0,1928,0.0
4,node_01_A,"((node_01_A, node_26_C))",1,0,1061,0.0
...,...,...,...,...,...,...
6848,node_27_C,"((node_27_C, node_11_B))",1,752,0,1.0
6849,node_27_C,"((node_27_C, node_01_B))",1,615,0,1.0
6850,node_27_C,"((node_27_C, node_15_C))",1,805,0,1.0
6851,node_27_C,"((node_27_C, node_12_A))",1,563,0,1.0


In [150]:
# 踢出common rows/参与优化的打包方案
fullplan_best_arranged = fullplan_best.loc[fullplan_best.merge(common_rows, indicator='i', how='left')['i'].apply(lambda x: x != 'both').values,:]
fullplan_merit_arranged = fullplan_merit.loc[fullplan_merit.merge(common_rows, indicator='i', how='left')['i'].apply(lambda x: x != 'both').values,:]
fullplan_worst_arranged = fullplan_worst.loc[fullplan_worst.merge(common_rows, indicator='i', how='left')['i'].apply(lambda x: x != 'both').values,:]


In [ ]:
#计算普通包 非零项 平均值

In [201]:
fullplan_worst_arranged[fullplan_worst_arranged['NormalRequest'] != 0]['NormalRequest'].mean()

639.0761968981793

In [200]:
fullplan_merit_arranged[fullplan_merit_arranged['NormalRequest'] != 0]['NormalRequest'].mean()

1865.5180265654649

In [198]:
fullplan_best_arranged[fullplan_best_arranged['NormalRequest'] != 0]['NormalRequest'].mean()

2403.912937347437

In [199]:
common_rows[common_rows['NormalRequest'] != 0]['NormalRequest'].mean()

3045.708270676692

In [ ]:
#计算快包 非零项 平均值

In [202]:
fullplan_worst_arranged[fullplan_worst_arranged['FastRequest'] != 0]['FastRequest'].mean()

282.2454663212435

In [203]:
fullplan_merit_arranged[fullplan_merit_arranged['FastRequest'] != 0]['FastRequest'].mean()

618.8829084041548

In [204]:
fullplan_best_arranged[fullplan_best_arranged['FastRequest'] != 0]['FastRequest'].mean()

926.7947368421053

In [205]:
common_rows[common_rows['FastRequest'] != 0]['FastRequest'].mean()

133.86426526717557

In [ ]:
# 快包占用额外打包流向数

In [211]:
sum(fullplan_best_arranged[fullplan_best_arranged['FastRequest'] != 0]['extra'])

130.0

In [213]:
sum(fullplan_merit_arranged[fullplan_merit_arranged['FastRequest'] != 0]['extra'])

412.0

In [214]:
sum(fullplan_worst_arranged[fullplan_worst_arranged['FastRequest'] != 0]['extra'])

1073.0

In [212]:
sum(common_rows[common_rows['FastRequest'] != 0]['extra'])

3854.0

In [175]:
# save to csv

fullplan_best['fullplan']=fullplan_best['fullplan'].apply(list)
fullplan_merit['fullplan'] = fullplan_merit['fullplan'].apply(list)
fullplan_worst['fullplan'] = fullplan_worst['fullplan'].apply(list)
fullplan_merit.to_csv('fullplan_merit.csv',index=False)
fullplan_worst.to_csv('fullplan_worst.csv',index=False)
fullplan_best.to_csv('fullplan_best.csv',index=False)
common_rows['fullplan']=common_rows['fullplan'].apply(list)
common_rows.to_csv('common_rows.csv',index=False)